In [13]:
from covid import utils
import numpy as np
import pandas as pd

In [30]:
individuals_path =  "/Users/sandernordeide/Epidemiologic-Vaccine-Strategies/results/GA_20210508235844/individuals/individuals_3.pkl"
score_path =        "/Users/sandernordeide/Epidemiologic-Vaccine-Strategies/results/GA_20210508235844/final_scores/final_score_3.pkl"
overview_path =     "/Users/sandernordeide/Epidemiologic-Vaccine-Strategies/results/GA_20210508235844/generation_overview.csv"

final_score = utils.read_pickle(score_path)
individuals = utils.read_pickle(individuals_path)
overview = pd.read_csv(overview_path)

In [31]:
for ID in final_score.keys():
    print(np.mean(final_score[ID]))

3495.2165017915336
3542.618018069722
3584.140941376204
3607.5620437107127
3676.620969674579
3673.660019359678
3697.784061010033
3705.60357333547
3696.7884710382264
3703.546457617359
3713.7591071919305
3732.983517811395
3752.1506272721467
3765.1583455334403
3756.4569043819415
3787.891508265271
3813.527980939724
3830.225702408637
3828.499517928723
3899.2670274642196
4017.426002770974
4889.716582211966
3542.967719281923


In [32]:
overview.sort_values('mean_score')

,generation,individual,mean_score
20,1,gen_0_004,3332.564053
21,1,gen_1_003,3378.872754
41,2,gen_0_004,3408.791354
22,1,gen_0_006,3451.338110
0,0,gen_0_004,3458.739189
...,...,...,...
61,2,gen_0_002,4032.587558
85,3,gen_0_001,4889.716582
19,0,gen_0_001,5008.671762
40,1,gen_0_001,5010.286814


In [33]:
for i in sorted(individuals, key=lambda x: x.mean_score):
    print(f"ID: {i.ID}")
    print(f"Strategy count: {i.strategy_count['D']}")
    print(f"Score: {i.mean_score}")
    print(f"Genes\n{i.genes}")
    print()

ID: gen_0_004
Strategy count: defaultdict(..., {})
Score: 3495.2165017915336
Genes
[[[0. 0. 0. 1.]
  [0. 0. 0. 1.]
  [0. 0. 0. 1.]
  [0. 0. 0. 1.]]

 [[0. 0. 0. 1.]
  [0. 0. 0. 1.]
  [0. 0. 0. 1.]
  [0. 0. 0. 1.]]

 [[0. 0. 0. 1.]
  [0. 0. 0. 1.]
  [0. 0. 0. 1.]
  [0. 0. 0. 1.]]]

ID: gen_1_003
Strategy count: defaultdict(..., {})
Score: 3542.618018069722
Genes
[[[0.   0.   0.25 0.75]
  [0.   0.   0.25 0.75]
  [0.   0.   0.25 0.75]
  [0.   0.   0.5  0.5 ]]

 [[0.   0.   0.25 0.75]
  [0.   0.   0.25 0.75]
  [0.   0.   0.   1.  ]
  [0.   0.   0.25 0.75]]

 [[0.   0.   0.25 0.75]
  [0.   0.2  0.2  0.6 ]
  [0.   0.   0.5  0.5 ]
  [0.   0.   0.25 0.75]]]

ID: gen_3_002
Strategy count: defaultdict(..., {})
Score: 3542.967719281923
Genes
[[[0.   0.   0.25 0.75]
  [0.   0.   0.25 0.75]
  [0.   0.   0.25 0.75]
  [0.   0.   0.   1.  ]]

 [[0.   0.   0.25 0.75]
  [0.   0.   0.25 0.75]
  [0.   0.   0.   1.  ]
  [0.   0.   0.   1.  ]]

 [[0.   0.   0.25 0.75]
  [0.   0.2  0.2  0.6 ]
  [0.   0.   0.